In [5]:
import pandas as pd

import pyarrow.parquet as pq
from pyarrow.parquet import ParquetFile
import pyarrow as pa 
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
import tqdm
from imblearn.under_sampling import CondensedNearestNeighbour
import datetime

# тут просто отбираем все лайки + случайным образом игноры в пропорции 1:15
# таких датасета было сформировано 2 с различными random_state

In [6]:
parquet_file = pq.ParquetFile('../../input_data/train.parquet')

batch_size = 10000000
res = pd.DataFrame()
count = 0

for rows in parquet_file.iter_batches(batch_size=batch_size):

    df = rows.to_pandas()     
  
     
    class0 = resample(df[df.target==0], replace=False, n_samples=df.target.value_counts()[1] * 14, random_state=0)
    data = pd.concat([class0, df[df.target==1]], axis=0).sort_values(by=['event_date']).reset_index().drop(columns=['index'])
   
    data.index = data.index + count
    
    tl = TomekLinks(sampling_strategy='majority')
    X, y = tl.fit_resample(data.drop(columns=['target', 'event_date']), data.target)
    
    X.index = tl.sample_indices_ + count
    y.index = tl.sample_indices_ + count
    
    count += df.shape[0]
    print(count)  
    
    X = pd.concat([X, data.event_date.iloc[tl.sample_indices_]], axis=1) 
    X = pd.concat([X, y], axis=1) 
        
    print(y.value_counts(), y.shape)
    res = pd.concat([res, X], axis=0) 

10000000
target
0    596264
1     44327
Name: count, dtype: int64 (640591,)
20000000
target
0    626631
1     46566
Name: count, dtype: int64 (673197,)
30000000
target
0    625333
1     46484
Name: count, dtype: int64 (671817,)
40000000
target
0    614045
1     45648
Name: count, dtype: int64 (659693,)
50000000
target
0    632345
1     47002
Name: count, dtype: int64 (679347,)
60000000
target
0    642471
1     47742
Name: count, dtype: int64 (690213,)
70000000
target
0    635234
1     47208
Name: count, dtype: int64 (682442,)
80000000
target
0    679111
1     50479
Name: count, dtype: int64 (729590,)
90000000
target
0    632767
1     47016
Name: count, dtype: int64 (679783,)
100000000
target
0    774808
1     57494
Name: count, dtype: int64 (832302,)
110000000
target
0    1249244
1      92224
Name: count, dtype: int64 (1341468,)
114741035
target
0    593407
1     43936
Name: count, dtype: int64 (637343,)


In [7]:
res.target.value_counts()

target
0    8301660
1     616126
Name: count, dtype: int64

In [8]:
res.to_csv('batch_resample_2_f.csv', sep=',', index=False)